In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

class SwarmAgent:
    def __init__(self, agent_id, position):
        self.agent_id = agent_id
        self.position = position

class FeedforwardNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FeedforwardNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

class SwarmController:
    def __init__(self, num_agents, environment_size, hidden_size, learning_rate):
        self.num_agents = num_agents
        self.environment_size = environment_size
        self.hidden_size = hidden_size
        self.learning_rate = learning_rate

        self.model = FeedforwardNN(environment_size[0]*environment_size[1], hidden_size, num_agents)
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

    def train(self, agent_data, labels, num_epochs):
        agent_data_tensor = torch.tensor(agent_data, dtype=torch.float)
        labels_tensor = torch.tensor(labels, dtype=torch.long)

        for epoch in range(num_epochs):
            self.optimizer.zero_grad()
            outputs = self.model(agent_data_tensor)
            loss = self.criterion(outputs, labels_tensor)
            loss.backward()
            self.optimizer.step()

    def get_action(self, agent):
        state = np.zeros(self.environment_size)
        state[tuple(agent.position)] = 1
        state_tensor = torch.tensor(state.flatten(), dtype=torch.float).unsqueeze(0)
        outputs = self.model(state_tensor)
        action = torch.argmax(outputs).item()
        return action

# Usage example
num_agents = 10
environment_size = (10, 10)
hidden_size = 64
learning_rate = 0.001

swarm_agents = []
for i in range(num_agents):
    position = np.random.randint(0, environment_size[0]), np.random.randint(0, environment_size[1])
    agent = SwarmAgent(i, position)
    swarm_agents.append(agent)

agent_data = [np.random.rand(environment_size[0]*environment_size[1]) for _ in range(num_agents)]
labels = np.random.randint(0, num_agents, size=num_agents)

swarm_controller = SwarmController(num_agents, environment_size, hidden_size, learning_rate)
swarm_controller.train(agent_data, labels, num_epochs=100)

# Get action for a specific agent
agent_id = 1
action = swarm_controller.get_action(swarm_agents[agent_id])
print(f"Action for Agent {agent_id}: {action}")


Action for Agent 1: 7


The SwarmAgent class represents an agent in the swarm, holding an agent ID and a position in the environment.

The FeedforwardNN class defines a feedforward neural network model. It consists of two fully connected layers with a ReLU activation function in between.

The SwarmController class represents the controller that uses the feedforward neural network for decision-making. It takes the number of agents, environment size, hidden layer size, and learning rate as input.

The SwarmController initializes a FeedforwardNN model with the specified input size, hidden size, and output size (number of agents). It also defines the criterion (cross-entropy loss) and optimizer (Adam optimizer).

The train method trains the model using the provided agent data and labels. It iterates over the specified number of epochs, calculates the loss, and updates the model's parameters.

The get_action method takes an agent as input and uses the model to predict the action for that agent based on its position in the environment.

In the usage example, a swarm of agents is created with random positions in the environment.

Random agent data and labels are generated for training the model.

An instance of SwarmController is created, passing the necessary parameters.

The train method is called to train the model using the agent data and labels.

The get_action method is called to retrieve the action for a specific agent.

The action for the agent is printed.

In summary, the code demonstrates the usage of a feedforward neural network model as a swarm controller. It trains the model using agent data and labels, allowing the model to predict actions for the swarm agents based on their positions in the environment.

